## Employee Wellness Pridiction

### Problem Statement
- This project is based on classification problem statement.
- Where the organization has lost one of it's most important employee.
- Now organization is conserned for the wellbeing of the other employees.

### Goal 
- To predict the wellbeing of the employee.
- Help organization in finding the employees in need.

### Dataset

These are the parameters provided by the company for prediction.
- Variables and their meaning:
    - Timestamp
    - Age: age of the employee.
    - Gender: Gender of the employee.
    - Country: Country of the employee.
    - state: If you live in the United States, which state or territory do you live in?
    - self_employed: Are you self-employed?
    - family_history: Do you have a family history of mental illness?
    - treatment: Does he or she really needs treatment.
    - work_interfere: If you have a mental health condition, do you feel that it interferes with your work?
    - no_employees: How many employees does your company or organization have?
    - remote_work: Do you work remotely (outside of an office) at least 50% of the time?
    - tech_company: Is your employer primarily a tech company/organization?
    - benefits: Does your employer provide mental health benefits?
    - care_options: Do you know the options for mental health care your employer provides?
    - wellness_program: Has your employer ever discussed mental health as part of an employee wellness program?
    - seek_help: Does your employer provide resources to learn more about mental health issues and how to seek help?
    - anonymity: Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources?
    - leave: How easy is it for you to take medical leave for a mental health condition?
    - mental_health_consequence: Do you think that discussing a mental health issue with your employer would have negative consequences?
    - phys_health_consequence: Do you think that discussing a physical health issue with your employer would have negative consequences?
    - coworkers: Would you be willing to discuss a mental health issue with your coworkers?
    - supervisor: Would you be willing to discuss a mental health issue with your direct supervisor(s)?
    - mental_health_interview: Would you bring up a mental health issue with a potential employer in an interview?
    - phys_health_interview: Would you bring up a physical health issue with a potential employer in an interview?
    - mental_vs_physical: Do you feel that your employer takes mental health as seriously as physical health?
    - obs_consequence: Have you heard of or observed negative consequences for coworkers with mental health conditions in your workplace?
    - comments: Any additional notes or comments.

### Data Collection

- Data is to be found on website.
- So we will download and store it localy first.

In [1]:
import os

In [2]:
%pwd

'd:\\ML\\xyz_employee_welness\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML\\xyz_employee_welness'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: list
    data_dir: list

In [7]:
from empWellness.constants import *
from empWellness.utils.common import read_yaml, create_directries
from empWellness.logging import logger
import requests

In [8]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directries([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            data_dir=config.data_dir,
        )

        return data_ingestion_config

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def ingest(self):
        try:
            logger.info("Ingestion started!")

            create_directries([self.config.root_dir])

            file_name = ["train.csv", "test.csv", "submission.csv"]

            for i in range(len(self.config.source_URL)):
                file_path = os.path.join(self.config.root_dir, self.config.data_dir[i], file_name[i])
                # print(file_path)

                if not os.path.exists(file_path):
                    file_dir = os.path.join(
                        self.config.root_dir, self.config.data_dir[i]
                    )
                    create_directries([file_dir])
                    logger.info(f"created dirctory: {file_dir[i]} for {file_name[i]}")

                    response = requests.get(self.config.source_URL[i])
                    raw_data = response.content.decode(
                        encoding="utf-8", errors="ignore"
                    )
                    logger.info(f"File: {file_name[i]} downloaded!")

                    with open(file_path, "w") as f:
                        f.write(raw_data)
                        logger.info(f"File: {file_name[i]} saved to {file_dir[i]}")
                        
                else:
                    logger.info(f"File: {file_name} already exists!")

        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.ingest()
except Exception as e:
    raise e